# Workshop CentraleSupélec - CeSIA

- Création : 02/2025 par [Nicolas Guillard](mailto:nicolas.guillar@securite-ia.fr) - bénévole au [CeSIA](https://www.securite-ia.fr/).

Créer en adaptant et complétant le projet [Générer des noms de villes et communes françaises](https://github.com/alxndrTL/villes) par [Alexandre TL](https://www.youtube.com/@alexandretl)


## Présentation du sujet et Plan

## Indications

Les éléments de ce TP :
- le présent carnet
- le répertoire `utils` et les fichiers contenus
- le fichier de données
- le répertoire `weights` contenant les poids des modèles utiles et ceux produits

### Pour cette partie :
Exécuter les différentes bloc de code successivement afin de découvrir le jeu de donnée et son traitement afin de produire des séquences pour le modèle Transformer.

Certains constats vous permettront d'apprécier les résultats obtenus dans les parties suivantes en connaissant la référence "vérité terrain".

Le jeu de données contient 36583 noms de commune française.

## Les modules et paramétrages globaux

Tous les modules nécessaires sont importés. A moins d'un besoin spécifique, il n'y aura pas besoin de modifier le bloc de code suivant.

In [ ]:
# Modules prédéfinis et tiers
import math
import datetime
from dataclasses import dataclass
from collections import Counter
from typing import Tuple

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.notebook import trange, tqdm
import torch

In [ ]:
#Modules créés pour le projet 
from utils import print_colore

### Device

Sélection du GPU selon l'environnement de travail

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

### Paramétrages

In [ ]:
# Retirer la limite du nombre maximal de lignes affichées dans un tableau pandas
pd.set_option('display.max_rows', None) 

In [ ]:
# Configurer le thème de seaborn
sns.set_theme(style="whitegrid")

In [ ]:
# Paramétrer les graines aléatoires
pth_rnd_gen = torch.Generator(device).manual_seed(42)

## Exploration des données

In [ ]:
df = pd.read_table("./villes.txt", header=None, names=["nom"])
display(df.head(20))

Affichons les premières informations structurelles :

In [ ]:
df.info()

### Quelques statistiques

#### Distribution de la longueur des noms

In [ ]:
# Calculer la longueur des chaînes de caractères dans la colonne "nom"
df['length'] = df['nom'].apply(len)

# Afficher la distribution de la longueur des chaînes de caractères
length_distribution = df['length'].value_counts().sort_index()
#print(length_distribution)

# Afficher la distribution sous forme d'un histogramme
plt.figure(figsize=(12, 8))
sns.barplot(x=length_distribution.index, y=length_distribution.values, hue=length_distribution.values, palette="coolwarm")
plt.xlabel('Longueur des chaînes de caractères')
plt.ylabel('Fréquence')
plt.title('Distribution de la longueur des chaînes de caractères')
plt.xticks(rotation=90)
plt.show()

In [ ]:
df['length'].describe()

#### Fréquences des caractères dans les noms

In [ ]:
# Concaténer toutes les chaînes de caractères de la colonne "nom"
all_chars = ''.join(df['nom'])

# Compter les occurrences de chaque caractère
char_counts = Counter(all_chars)

# Convertir le résultat en dataframe pour une meilleure lisibilité
char_freq_df = pd.DataFrame(
    char_counts.items(), columns=['Caractère', 'Fréquence']
    ).sort_values(by='Fréquence', ascending=False)

print("Nombre de caractères distincts :", len(char_freq_df))
display(char_freq_df)

Taux de fréquence par rapport aux (dix premières) positions dans la chaîne de caractères

In [ ]:
# Limiter la longueur des noms à 10 caractères
df['nom_limited'] = df['nom'].str[:10]

# Initialiser un dictionnaire pour stocker les fréquences des caractères par position
position_char_freq = {i: Counter() for i in range(10)}

# Remplir le dictionnaire avec les fréquences des caractères par position
for name in df['nom_limited']:
    for i, char in enumerate(name):
        position_char_freq[i][char] += 1

# Convertir le dictionnaire en dataframe pour une meilleure lisibilité
position_char_freq_df = pd.DataFrame(position_char_freq).fillna(0).astype(int)

# Limiter aux 15 premiers caractères les plus fréquents
top_chars = position_char_freq_df.sum(axis=1).sort_values(ascending=False).head(15).index
position_char_freq_df = position_char_freq_df.loc[top_chars]

# Calculer le taux de présence par position
position_char_rate_df = position_char_freq_df.div(position_char_freq_df.sum(axis=0), axis=1) * 100

# Visualiser les taux de présence avec une carte de chaleur
plt.figure(figsize=(12, 8))
sns.heatmap(position_char_rate_df, annot=True, fmt=".2f", cmap="coolwarm", cbar=True)
plt.xlabel('Position')
plt.ylabel('Caractère')
plt.title('Taux de présence des caractères par position (limité aux 10 premières positions)')
plt.show()

#### Fréquence des composants de nom

In [ ]:
# Séparer les chaînes de caractères par "-" ou " " et les concaténer
element_separator = "-' "
all_elements = ' '.join(df['nom'].str.replace(f"[{element_separator}]", " ", regex=True).values).split()

# Compter les occurrences de chaque élément
element_counts = Counter(all_elements)

# Convertir le résultat en dataframe pour une meilleure lisibilité
element_freq_df = pd.DataFrame(element_counts.items(), columns=['Élément', 'Fréquence']).sort_values(by='Fréquence', ascending=False)

print(f"Nombre total de composants distincts : {len(element_freq_df)}")
display(element_freq_df.sample(15).sort_values(by='Fréquence', ascending=False))

# Filtrer les éléments dont la fréquence est supérieure à 1
element_freq_sup_1_df = element_freq_df[element_freq_df['Fréquence'] > 1]

# Afficher le nombre total d'éléments associés
print(f"Nombre total de composants présents plus d'une fois : {len(element_freq_sup_1_df)}")
display(element_freq_sup_1_df.head(15))


Combien de noms de communes sont composées ?
Quelle est la distribution du nombre de composants ?

In [ ]:
# Calculer le nombre de composants pour chaque nom
df['num_components'] = df['nom'].apply(lambda x: len([comp for comp in x if comp in element_separator]) + 1)

# Afficher la distribution du taux de fréquence du nombre de composants
component_distribution = df['num_components'].value_counts().sort_index()
component_distribution_rate = component_distribution / len(df) * 100
display(component_distribution.to_frame())

# Afficher la distribution sous forme d'un histogramme
plt.figure(figsize=(12, 8))
sns.barplot(x=component_distribution_rate.index, y=component_distribution_rate.values, hue=component_distribution_rate.values, palette="coolwarm")
for i in range(len(component_distribution_rate)):
    plt.text(i, component_distribution_rate.values[i] + 0.5, f'{component_distribution_rate.values[i]:.3f}%', ha='center')
plt.xlabel('Nombre de composants')
plt.ylabel('Fréquence')
plt.title('Distribution de la fréquence du nombre de composants')
plt.xticks(rotation=90)
plt.show()

## Dataset

Définition de la classe fournissant les données

In [ ]:
PAD = "<pad>" # Padding
SOS = "<SOS>" # Start Of Sequence
EOS = "<EOS>" # End Of Sequence

class CityNameDataset():
    def __init__(self, file_name: str = "villes.txt", split_rate: float =0.9, device="cpu"):

        # chargement des données
        fichier = open(file_name)
        donnees = fichier.read()
        villes = donnees.replace('\n', ',').split(',')
        # mise à l'écart des villes avec un nom de longueur inférieure à 3
        self.villes = [ville for ville in villes if len(ville) > 2]

        # création du vocabulaire
        self.vocabulaire = sorted(list(set(''.join(villes))))
        self.vocabulaire = [PAD, SOS, EOS] + self.vocabulaire
        # <SOS> et <EOS> sont ajoutés respectivement au début et à la fin de chaque séquence
        # <pad> est utilisé pour faire en sorte que toutes les séquences aient la même longueur

        # pour convertir char <-> int
        self.char_to_int = {}
        self.int_to_char = {}

        for (c, i) in tqdm(zip(self.vocabulaire, range(len(self.vocabulaire))), desc="creating vocabulary", total=len(self.vocabulaire)):
            self.char_to_int[c] = i
            self.int_to_char[i] = c

        num_sequences = len(villes)
        self.max_len = max([len(ville) for ville in villes]) + 2 # <SOS> et <EOS>

        X = torch.zeros((num_sequences, self.max_len), dtype=torch.int32, device=device)

        # création des séquences
        for i in trange(num_sequences, desc="creatind dataset"):
            X[i] = torch.tensor([self.char_to_int[SOS]] +
                                [self.char_to_int[c] for c in villes[i]] +
                                [self.char_to_int[EOS]] +
                                [self.char_to_int[PAD]] * (self.max_len - len(villes[i]) - 2))

        # jeu de données d'entrainement et de validation
        n_split = int(split_rate * X.shape[0])

        idx_permut = torch.randperm(X.shape[0])
        idx_train, _ = torch.sort(idx_permut[:n_split])
        idx_val, _ = torch.sort(idx_permut[n_split:])

        self.X_train = X[idx_train]
        self.X_val = X[idx_val]

    def get_batch(self, batch_size: int, split : str="val", device=None) -> torch.Tensor:
        assert split in ["train", "val"], f"split ({split}) should be 'train' or 'val'." 

        data = self.X_train if split == 'train' else self.X_val

        idx = torch.randint(low=int(batch_size/2), high=int(data.shape[0]-batch_size/2), size=(1,), dtype=torch.int32).item()

        batch = data[int(idx-batch_size/2):int(idx+batch_size/2)]
        X = batch[:, :-1] # (B, S=max_len-1) : (max(len("nom")) + 2) - 1
        Y = batch[:, 1:] # (B, S=max_len-1)
        if device:
            X = X.to(device)
            Y = Y.to(device)
        return X, Y.long()
    
    def cast_char_to_int(self, sequence: list[str]) -> list[int]:
        return [self.char_to_int[c] for c in sequence]
    
    def cast_int_to_char(self, sequence: list[int]) -> list[str]:
        return [self.int_to_char[i] for i in sequence]
    
    def to_string(self, sequence: list[int]) -> str:
        return "".join([self.int_to_char[i] for i in sequence if i > 2])

Test du dataset

In [ ]:
dataset = CityNameDataset(device=device) # toute la partie données de 2_mlp.py a été encapsulée dans l'objet Dataset

In [ ]:
x, Y = batch = dataset.get_batch(batch_size=2)
print("> X (ids):", x.to("cpu"), sep="\n")
print("> caractères correspondants pour la première séquence :", dataset.cast_int_to_char(x[0].to("cpu").tolist()), sep="\n")
print("> nom :", dataset.to_string(x.to("cpu")[0].tolist()))